In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
env = "dev"

In [3]:
from ml_feature_store.common.spark_context import create_spark_context
spark = create_spark_context()

In [ ]:
from ml_feature_store.feature_tables import ft_recipe_ingredients_configs
table_config = ft_recipe_ingredients_configs
table_config

In [7]:
primary_keys = table_config.primary_keys
feature_table_name_full = table_config.feature_table_name
add_pk_sql = f"""
    ALTER TABLE {feature_table_name_full}
    ADD CONSTRAINT {primary_keys} PRIMARY KEY(
    {', '.join(primary_keys)});
"""

In [ ]:
print(add_pk_sql)

In [ ]:
table_config.primary_keys

In [5]:
from ml_feature_store.common.data import get_data_from_catalog
df_recipe_ingredients = get_data_from_catalog(
    spark=spark,
    env=env,
    table_name=table_config.dbt_model_name,
    schema="mlgold",
    is_convert_to_pandas=True,
)

In [6]:
from ml_feature_store.ft_recipe_ingredients.feature_table_builder import get_ingredient_data
df_distinct_ingredients = get_ingredient_data(
    spark=spark,
    env="dev",
    schema="mlgold"
)

In [7]:
from ml_feature_store.ft_recipe_ingredients.text_feature_generator import add_nouns_only_and_lemmatized_columns
df_lemmatized_nouns = add_nouns_only_and_lemmatized_columns(
    df_distinct_ingredients=df_distinct_ingredients,
    text_column_name="generic_ingredient_name",
    nouns_only_column_name="generic_ingredient_name_nouns_only",
    lemmatized_column_name="generic_ingredient_lemmatized",
)

In [8]:
from ml_feature_store.ft_recipe_ingredients.text_feature_generator import get_generic_name_lemmatized_nouns

df_recipe_ingredients = get_generic_name_lemmatized_nouns(
    df_recipe_ingredients=df_recipe_ingredients,
    df_distinct_ingredients=df_distinct_ingredients,
    distinct_ingredient_text_col_name="generic_ingredient_name",
    distinct_ingredient_id_column_name="generic_ingredient_id",
    nouns_only_column_name="generic_ingredient_name_nouns_only",
    lemmatized_column_name="generic_ingredient_name_lemmatized_nouns_only"
)

In [9]:
from ml_feature_store.ft_recipe_ingredients.configs.categories import categories
from ml_feature_store.ft_recipe_ingredients.category_features_generator import add_category_features
df_recipe_ingredients_with_categories = add_category_features(
    df_recipe_ingredients=df_recipe_ingredients,
    category_mappings=categories,
    binary_column_prefix="has_"
)